In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
with open('../real_data/Poisson_model_data.json', 'r') as f: br_23_data = json.load(f)

In [3]:
class championship:
    def __init__(self, data) -> None:
        self.num_games = data['num_games']
        self.num_teams = data['num_teams']
        self.team1_list = data['team1']
        self.team2_list = data['team2']
        self.goals_team1 = data['goals_team1']
        self.goals_team2 = data['goals_team2']
        self.teams_names = data['teams']
        self.games = self.make_games_table()

    def make_games_table(self):
        teams = pd.DataFrame(self.teams_names, columns=['team'])
        teams['id'] = teams.index + 1

        games = dict()
        games['home id'] = self.team1_list
        games['home score'] = self.goals_team1
        games['away score'] = self.goals_team2
        games['away id'] = self.team2_list
        games = pd.DataFrame(games)
        games['game id'] = games.index + 1

        games = games.merge(teams, left_on='home id', right_on='id')
        games.rename({'team': 'home team'}, inplace=True, axis=1)
        games.drop(['id', 'home id'], inplace=True, axis=1)

        games = games.merge(teams, left_on='away id', right_on='id')
        games.rename({'team': 'away team'}, inplace=True, axis=1)
        games.drop(['id', 'away id'], inplace=True, axis=1)

        games = games[['game id', 'home team', 'home score', 'away score', 'away team']]
        games.sort_values('game id', inplace=True, ignore_index=True)
        
        return games
    
    def create_empty_standings_table(self):
        standings = dict()
        standings['team'] = self.teams_names
        standings['points'] = [0] * self.num_teams
        standings['wins'] = [0] * self.num_teams
        standings['draws'] = [0] * self.num_teams
        standings['goals for'] = [0] * self.num_teams
        standings['goals against'] = [0] * self.num_teams
        standings['goals difference'] = [0] * self.num_teams
        
        return standings
    
    def get_standings(self):
        standings = pd.DataFrame(self.standings)
        standings.sort_values(['points', 'wins', 'goals difference', 'goals for'],
                              ascending=[False, False, False, False],
                              inplace=True, ignore_index=True)

        return standings

    def calculate_standing_until_round(self, round_number):
        games = self.games[self.games['game id'] <= round_number * 10]
        self.standings = self.create_empty_standings_table()
        for i in games.index:
            self.update_standings_with_game(games.loc[i])

        return self.get_standings()

    def update_standings_with_game(self, game):
        home_index = self.standings['team'].index(game['home team'])
        away_index = self.standings['team'].index(game['away team'])

        home_goals = game['home score']
        away_goals = game['away score']

        if home_goals > away_goals:
            self.standings['wins'][home_index] += 1
            self.standings['points'][home_index] += 3
        elif home_goals < away_goals:
            self.standings['wins'][away_index] += 1
            self.standings['points'][away_index] += 3
        else:
            self.standings['draws'][home_index] += 1
            self.standings['draws'][away_index] += 1
            self.standings['points'][home_index] += 1
            self.standings['points'][away_index] += 1

        self.standings['goals for'][home_index] += home_goals
        self.standings['goals for'][away_index] += away_goals

        self.standings['goals against'][away_index] += home_goals
        self.standings['goals against'][home_index] += away_goals

        self.standings['goals difference'][home_index] += home_goals - away_goals
        self.standings['goals difference'][away_index] += away_goals - home_goals


In [4]:
br_23 = championship(br_23_data)
br_23.calculate_standing_until_round(38)

,team,points,wins,draws,goals for,goals against,goals difference
0,Palmeiras / SP,70,20,10,64,33,31
1,Grêmio / RS,68,21,5,63,56,7
2,Atlético Mineiro / MG,66,19,9,52,32,20
3,Flamengo / RJ,66,19,9,56,42,14
4,Botafogo / RJ,64,18,10,58,37,21
5,Red Bull Bragantino / SP,62,17,11,49,35,14
6,Fluminense / RJ,56,16,8,51,47,4
7,Athletico Paranaense / PR,56,14,14,51,43,8
8,Internacional / RS,55,15,10,46,45,1
9,Fortaleza / CE,54,15,9,45,44,1
